In [111]:
## Packages
##Imports
import pandas as pd
import transformers
import openpyxl
import numpy
import torch
import os
import numpy as np
import shutil
import sys
import tqdm.notebook as tq
from collections import defaultdict
import time

import torch
import torch.nn as nn

# Does augmenting classifiers with structured data help?
# Do it for both the baseline classical ML and transformers then compare
# Try it with just structured data [0s and 1s]

In [112]:
torch.cuda.is_available()

True

In [113]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060 Ti'

In [114]:
## Reading in the PSYTar data set and parsing it

## Use sentence_labelling sheet

fileName=".\ONLINE_FORA\PsyTAR_dataset.xlsx"
data=pd.ExcelFile(fileName)
sheets={}
for sheet in data.sheet_names:
    sheets[sheet]=data.parse(sheet)

## Remove the first two sheets (License and read_me)
sheets.pop('License',None)
sheets.pop('read_me',None)

## This will print out the sheet names for the whole excel
for sheet in sheets.keys():
    print(f"Sheet Name: {sheet}")

#To access a sheet, perform sheet['Sheet_Name']; e.g., sheets['Sample']

Sheet Name: Sample
Sheet Name: Sentence_Labeling
Sheet Name: ADR_Identified
Sheet Name: ADR_Mapped
Sheet Name: WD_Identified
Sheet Name: WD-Mapped 
Sheet Name: SSI_Identified
Sheet Name: SSI_Mapped
Sheet Name: DI_Identified
Sheet Name: DI_Mapped


In [115]:
## Vectorize data into TF-IDF and preprocess data

import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer


def preprocess(text):
    text=str(text)
    tokens=nltk.word_tokenize(text.lower())
    tokens_clean=[t for t in tokens if (t not in stop_words) and (t not in punctuations)]
    return ' '.join(tokens_clean)
    
stop_words=nltk.corpus.stopwords.words('english')
punctuations=string.punctuation

df=data.parse('Sentence_Labeling')
df.drop(df.tail(1).index,inplace=True)
df['drug_id']=df['drug_id'].str.lower()
df['drug_name']=df['drug_id'].str.replace(r'\.\d+','',regex=True)
df['sentences']=df['sentences'].apply(preprocess)
df.fillna(0,inplace=True)
unique_drug_count=df['drug_name'].nunique()

tfidf=TfidfVectorizer()
tfidfSentences=tfidf.fit_transform(df['sentences'])

print(tfidfSentences)

print('Number of Drug Types:',unique_drug_count)
print(df)

## ADR: adverse drug reaction
## WD: withdrawal symptom
## EF: effective
## INF: ineffective
## SSI: Sign/symptom/illness - if report contains explicit SSI that patient experienced that are not a result of the drug
## DI: drug indication - shows SSI that explicitly mentioned as being resolved because of drug consumption

  (0, 2183)	0.3560733570424169
  (0, 2858)	0.5335210557775061
  (0, 2980)	0.33695305203929327
  (0, 4715)	0.3577130194209486
  (0, 4193)	0.36113778160835414
  (0, 2071)	0.2627216248356759
  (0, 5156)	0.2390135798252401
  (0, 1829)	0.30083291559032393
  (1, 2765)	0.4585168729312828
  (1, 1351)	0.8886857021677111
  (2, 2263)	0.2708094866549267
  (2, 4586)	0.4316846093641784
  (2, 5071)	0.44358904667953863
  (2, 4827)	0.2570546169681927
  (2, 3078)	0.23875893121623276
  (2, 4151)	0.3207489298964512
  (2, 1492)	0.29911785906957045
  (2, 1201)	0.35264280744022497
  (2, 4287)	0.3220996041881179
  (3, 4012)	0.534375375933816
  (3, 3326)	0.5729355939201095
  (3, 1018)	0.39488574070665183
  (3, 1237)	0.3054771887014283
  (3, 5)	0.3700495936825698
  (4, 4801)	0.3601630582861626
  :	:
  (6006, 3810)	0.266723700884449
  (6006, 278)	0.30278988091992165
  (6006, 4033)	0.24160419308016043
  (6006, 1558)	0.24160419308016043
  (6006, 2821)	0.21617383387506503
  (6006, 3470)	0.20987895271102364
  (6006,

In [116]:
## This is just the plain sentences -- Task 1 Feature 1
from sklearn.model_selection import train_test_split

df['DI'] = pd.to_numeric(df['DI'], errors='coerce')

newDf=df
newDf['DI'].fillna(0, inplace=True)
newDf.drop(columns=["id", "comment_id", "drug_id", "sentence_index", "Findings", "others", "rating", "category","drug_name"], axis=1, inplace=True)

# x = df['sentences'].values
# y = df[['ADR', 'WD', 'EF', 'INF', 'SSI', 'DI']].values
# y = np.nan_to_num(y, nan=0).astype(int)

# Split data
df_train, df_temp = train_test_split(newDf, random_state=42, test_size=0.55, shuffle=True)
df_valid, df_test = train_test_split(df_temp, random_state=42, test_size=(1/11), shuffle=True)



C:\Users\julev\AppData\Local\Temp\ipykernel_26664\1541660150.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  newDf['DI'].fillna(0, inplace=True)


In [117]:
print('total length=',len(df))
print('%len of training set=',len(df_train)/len(df)*100)
print('%len of validation set=',len(df_valid)/len(df)*100)
print('%len of test set=',len(df_test)/len(df)*100)

total length= 6009
%len of training set= 44.99916791479448
%len of validation set= 49.99167914794475
%len of test set= 5.009152937260775


In [118]:
# # This one uses the drug_name with the sentences -- Task 1 Feature 2

# from sklearn.model_selection import train_test_split

# df['DI'] = pd.to_numeric(df['DI'], errors='coerce')
# newDf=df
# newDf['DI'].fillna(0, inplace=True)
# newDf['drug_sentences'] = df['drug_name'] + ' ' + df['sentences']
# newDf.drop(columns=["id", "comment_id", "drug_id", "sentence_index", "Findings", "others", "rating", "category","drug_name","sentences"], axis=1, inplace=True)
# # x = df['drug_sentences'].values
# # y = df[['ADR', 'WD', 'EF', 'INF', 'SSI', 'DI']].values
# # y = np.nan_to_num(y, nan=0).astype(int)

# # Split data
# df_train, df_temp = train_test_split(newDf, random_state=42, test_size=0.55, shuffle=True)
# df_valid, df_test = train_test_split(df_temp, random_state=42, test_size=(1/11), shuffle=True)



In [119]:
print(newDf)

                                              sentences  ADR   WD   EF  INF  \
0     extreme weight gain short-term memory loss hai...  1.0  0.0  0.0  0.0   
1                                      detoxing lexapro  0.0  0.0  0.0  0.0   
2     slowly cut dosage several months took vitamin ...  0.0  0.0  0.0  0.0   
3                          10 days completely omg rough  0.0  0.0  0.0  0.0   
4     flu-like symptoms dizziness major mood swings ...  0.0  1.0  0.0  0.0   
...                                                 ...  ...  ...  ...  ...   
6004                                increase dosage yet  0.0  0.0  0.0  0.0   
6005           'm hoping able stay 75 mgs long possible  0.0  0.0  0.0  0.0   
6006  reading withdrawals little scarey like said pe...  0.0  0.0  0.0  0.0   
6007  effexor made huge difference life come experie...  0.0  0.0  1.0  0.0   
6008              would small price pay able enjoy life  0.0  0.0  1.0  0.0   

      SSI   DI  
0     0.0  0.0  
1     0.0  0.0  


In [120]:
print(f"Train: {df_train.shape}, Test: {df_test.shape}, Valid: {df_valid.shape}")

Train: (2704, 7), Test: (301, 7), Valid: (3004, 7)


In [121]:
# Hyperparameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
TEST_BATCH_SIZE = 8
EPOCHS = 15
LEARNING_RATE = 1e-05
THRESHOLD = 0.5 # threshold for the sigmoid

In [122]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

c:\Users\julev\anaconda3\envs\cudaEnv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [123]:
#df_train['sentences']

In [124]:
class CustomDatasetTask1(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len, target_list):
        self.tokenizer = tokenizer
        self.df = df
        self.title = list(df['sentences'])
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index]),
            'title': title
        }
     
     
class CustomDatasetTask2(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len, target_list):
        self.tokenizer = tokenizer
        self.df = df
        self.title = list(df['drug_sentences'])
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index]),
            'title': title
        }
     

In [125]:
target_list = list(['ADR', 'WD', 'EF', 'INF', 'SSI', 'DI'])
target_list

['ADR', 'WD', 'EF', 'INF', 'SSI', 'DI']

In [126]:
#change this to CustomDatasetTask_ for feature _
train_dataset = CustomDatasetTask1(df_train, tokenizer, MAX_LEN, target_list)
valid_dataset = CustomDatasetTask1(df_valid, tokenizer, MAX_LEN, target_list)
test_dataset = CustomDatasetTask1(df_test, tokenizer, MAX_LEN, target_list)

In [127]:
next(iter(train_dataset))

{'input_ids': tensor([  101,  3635,  5114, 22356,  2606,  3279,  3637, 16915,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [128]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset, 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

test_data_loader = torch.utils.data.DataLoader(test_dataset, 
    batch_size=TEST_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)
     

In [129]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 6)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()

# # Freezing BERT layers: (tested, weaker convergence)
# for param in model.bert_model.parameters():
#     param.requires_grad = False

model.to(device)

c:\Users\julev\anaconda3\envs\cudaEnv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [130]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [131]:
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(), lr = 1e-5)    

c:\Users\julev\anaconda3\envs\cudaEnv\Lib\site-packages\transformers\optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [132]:
def train_model(training_loader, model, optimizer):

    losses = []
    correct_predictions = 0
    num_samples = 0
    # set model to training mode (activate droput, batch norm)
    model.train()
    # initialize the progress bar
    loop = tq.tqdm(enumerate(training_loader), total=len(training_loader), 
                      leave=True, colour='steelblue')
    for batch_idx, data in loop:
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        # forward
        outputs = model(ids, mask, token_type_ids) # (batch,predict)=(32,8)
        loss = loss_fn(outputs, targets)
        losses.append(loss.item())
        # training accuracy, apply sigmoid, round (apply thresh 0.5)
        outputs = torch.sigmoid(outputs).cpu().detach().numpy().round()
        targets = targets.cpu().detach().numpy()
        correct_predictions += np.sum(outputs==targets)
        num_samples += targets.size   # total number of elements in the 2D array

        # backward
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        # grad descent step
        optimizer.step()

        # Update progress bar
        #loop.set_description(f"")
        #loop.set_postfix(batch_loss=loss)

    # returning: trained model, model accuracy, mean loss
    return model, float(correct_predictions)/num_samples, np.mean(losses)

In [133]:
def eval_model(validation_loader, model, optimizer):
    losses = []
    correct_predictions = 0
    num_samples = 0
    # set model to eval mode (turn off dropout, fix batch norm)
    model.eval()

    with torch.no_grad():
        for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            losses.append(loss.item())

            # validation accuracy
            # add sigmoid, for the training sigmoid is in BCEWithLogitsLoss
            outputs = torch.sigmoid(outputs).cpu().detach().numpy().round()
            targets = targets.cpu().detach().numpy()
            correct_predictions += np.sum(outputs==targets)
            num_samples += targets.size   # total number of elements in the 2D array

    return float(correct_predictions)/num_samples, np.mean(losses)

In [134]:
torch.cuda.empty_cache()
history = defaultdict(list)
best_accuracy = 0

model_save_path = './model2_task1_feature1_model(unfrozen)'
start_time = time.time()

# for epoch in range(1, EPOCHS+1):
#     print(f'Epoch {epoch}/{EPOCHS}')
#     model, train_acc, train_loss = train_model(train_data_loader, model, optimizer)
#     val_acc, val_loss = eval_model(val_data_loader, model, optimizer)

#     print(f'train_loss={train_loss:.4f}, val_loss={val_loss:.4f} train_acc={train_acc:.4f}, val_acc={val_acc:.4f}')

#     history['train_acc'].append(train_acc)
#     history['train_loss'].append(train_loss)
#     history['val_acc'].append(val_acc)
#     history['val_loss'].append(val_loss)

#     if val_acc > best_accuracy:
#         if not os.path.exists(model_save_path):
#             os.makedirs(model_save_path)
#         torch.save(model.state_dict(), os.path.join(model_save_path, 'model_state.bin'))
#         print(f"Model saved to {model_save_path}")
#         best_accuracy = val_acc
         
# end_time = time.time()
# total_train_time = end_time - start_time

# print(f"Total training time: {total_train_time:.2f} seconds")
# torch.cuda.empty_cache()

In [135]:
import matplotlib.pyplot as plt

# plt.rcParams["figure.figsize"] = (10,7)
# plt.plot(history['train_acc'], label='train accuracy')
# plt.plot(history['val_acc'], label='validation accuracy')
# plt.title('Training history')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend()
# plt.ylim([0, 1])
# plt.grid()

In [136]:
torch.cuda.empty_cache()

In [137]:
# Loading pretrained model (best model)
model_load_path = './model2_task1_feature1_model(unfrozen)'
model = BERTClass()
model.load_state_dict(torch.load(os.path.join(model_load_path, 'model_state.bin')))
model = model.to(device)

In [138]:
test_acc, test_loss = eval_model(test_data_loader, model, optimizer)

In [139]:
test_acc

0.9839424141749723

In [140]:
from sklearn.metrics import confusion_matrix, classification_report
     
def get_predictions(model, data_loader):
    """
    Outputs:
      predictions - 
    """
    model = model.eval()
    
    titles = []
    predictions = []
    prediction_probs = []
    target_values = []

    with torch.no_grad():
      for data in data_loader:
        title = data["title"]
        ids = data["input_ids"].to(device, dtype = torch.long)
        mask = data["attention_mask"].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data["targets"].to(device, dtype = torch.float)
        
        outputs = model(ids, mask, token_type_ids)
        # add sigmoid, for the training sigmoid is in BCEWithLogitsLoss
        outputs = torch.sigmoid(outputs).detach().cpu()
        # thresholding at 0.5
        preds = outputs.round()
        targets = targets.detach().cpu()

        titles.extend(title)
        predictions.extend(preds)
        prediction_probs.extend(outputs)
        target_values.extend(targets)
    
    predictions = torch.stack(predictions)
    prediction_probs = torch.stack(prediction_probs)
    target_values = torch.stack(target_values)
    
    return titles, predictions, prediction_probs, target_values

In [141]:
titles, predictions, prediction_probs, target_values = get_predictions(model, test_data_loader)

In [142]:
predictions.numpy()

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]], dtype=float32)

In [143]:
print(f"titles:{len(titles)} \npredictions:{predictions.shape} \nprediction_probs:{prediction_probs.shape} \ntarget_values:{target_values.shape}")

titles:301 
predictions:torch.Size([301, 6]) 
prediction_probs:torch.Size([301, 6]) 
target_values:torch.Size([301, 6])


In [144]:
print(classification_report(target_values, predictions, target_names=target_list))

              precision    recall  f1-score   support

         ADR       0.95      0.97      0.96       108
          WD       0.88      0.83      0.86        18
          EF       0.96      0.89      0.92        55
         INF       1.00      0.92      0.96        13
         SSI       0.92      0.97      0.94        34
          DI       1.00      0.89      0.94        27

   micro avg       0.95      0.93      0.94       255
   macro avg       0.95      0.91      0.93       255
weighted avg       0.95      0.93      0.94       255
 samples avg       0.61      0.60      0.60       255



c:\Users\julev\anaconda3\envs\cudaEnv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\julev\anaconda3\envs\cudaEnv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\julev\anaconda3\envs\cudaEnv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [145]:
print(df_test)

                                              sentences  ADR   WD   EF  INF  \
2018  allow think one subject time previous 9,846 th...  0.0  0.0  1.0  0.0   
5762                   since power chair almost nothing  0.0  0.0  0.0  0.0   
5715                         dizziness standing bending  1.0  0.0  0.0  0.0   
1870  started taking 100mg went 50mg due headaches a...  1.0  0.0  0.0  0.0   
5142                            3 weeks stopped working  0.0  0.0  0.0  1.0   
...                                                 ...  ...  ...  ...  ...   
1894                                             sweats  1.0  0.0  0.0  0.0   
2240    zoloft made difference life 'm longer depressed  0.0  0.0  1.0  0.0   
2677                               first week two bliss  0.0  0.0  1.0  0.0   
4411                  currently take 40 mg cymbalta day  0.0  0.0  0.0  0.0   
191                               'm serious pain right  1.0  0.0  0.0  0.0   

      SSI   DI  
2018  1.0  1.0  
5762  0.0  0.0  


In [146]:
from sklearn.metrics import multilabel_confusion_matrix
true_test_labels = df_test[['ADR', 'WD', 'EF', 'INF', 'SSI', 'DI']].to_numpy(dtype=np.float32)
print(true_test_labels)

[[0. 0. 1. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]


In [147]:
confusion_matrixes=multilabel_confusion_matrix(true_test_labels,predictions.numpy())
class_names = ['ADR', 'WD', 'EF', 'INF', 'SSI', 'DI']

for idx, class_name in enumerate(class_names):
    cm = confusion_matrixes[idx]
    print(f"Confusion matrix for class '{class_name}':\n{cm}\n")
    report = classification_report(true_test_labels[:, idx], predictions[:, idx], target_names=[f'{class_name}_0', f'{class_name}_1'])
    print(report)

Confusion matrix for class 'ADR':
[[188   5]
 [  3 105]]

              precision    recall  f1-score   support

       ADR_0       0.98      0.97      0.98       193
       ADR_1       0.95      0.97      0.96       108

    accuracy                           0.97       301
   macro avg       0.97      0.97      0.97       301
weighted avg       0.97      0.97      0.97       301

Confusion matrix for class 'WD':
[[281   2]
 [  3  15]]

              precision    recall  f1-score   support

        WD_0       0.99      0.99      0.99       283
        WD_1       0.88      0.83      0.86        18

    accuracy                           0.98       301
   macro avg       0.94      0.91      0.92       301
weighted avg       0.98      0.98      0.98       301

Confusion matrix for class 'EF':
[[244   2]
 [  6  49]]

              precision    recall  f1-score   support

        EF_0       0.98      0.99      0.98       246
        EF_1       0.96      0.89      0.92        55

    accura